# 因子模型

## 开篇：因子模型解决什么问题、对谁有用

在实务里，因子模型首先用于**业绩归因**：把一只股票、一个组合或一只基金在某一段时间的**超额收益**，分解成几部分——一部分来自**广泛而稳定地影响许多资产的共同影响因素**（如“大盘整体涨跌”“小盘风格相对大盘”“价值相对成长”等），另一部分是该资产或该管理人特有的成分（策略选择、择时或纯噪声）。  
这样做的目的至少有三类：

* **解释**：回答“收益从哪里来”。例如，基金近期表现是否主要因为大盘走强，还是因为明显的价值（或小盘）倾向。  
* **评价**：区分“因子带来的正常回报”和“剥离因子后仍剩下的平均额外收益”（常记为 $\alpha$），从而更公平地评价管理能力。  
* **管理**：根据对因子“敏感度/斜率”的估计，控制或配置风格暴露，进行风险预算与再平衡（例如降低对单一风格的过度依赖）。

### 因子模型的基本思想

在一段时间尺度上，资产的期望收益可以写成“无风险利率 + 若干**共同影响因素**的线性组合 + 个体特有部分”。线性表达为：

$$
E[R_i] = R_f + \beta_{i1}\lambda_1 + \beta_{i2}\lambda_2 + \cdots + \beta_{ik}\lambda_k.
$$

这里，$\beta_{ij}$ 可以理解为**斜率/敏感度**：当第 $j$ 个因素的回报上升 $1$ 个单位时，资产 $i$ 的回报平均会随之变化多少；$\lambda_j$ 是该因素在样本期内对应的平均“回报补偿”。实际估计时，常用时间序列回归得到各 $\beta$，再用样本均值近似各因素的溢价。

### CAPM：以“大盘敏感度”解释超额收益

**思想**：如果把整个市场看作一只尽可能分散、按市值加权的“超大指数”，那么一只资产的超额收益，主要由它**随大盘一起涨跌的幅度**（斜率 $\beta$）决定。  
**回归形式**：

$$
R_{i,t}-R_{f,t} = \alpha_i + \beta_{i,M}(R_{M,t}-R_{f,t}) + \varepsilon_{i,t}.
$$

* $\beta_{i,M}$：对“大盘超额收益”的**斜率/敏感度**；可读作“大盘每变动 $1\%$，该资产平均变动 $\beta\%$”。  
* $\alpha_i$：在控制了大盘影响后仍然存在的**平均额外收益**；若 CAPM 完全成立，应接近 $0$。  
* 估计中通常使用月度总回报（含分红再投），并用 Newey–West 标准误修正自相关与异方差。

### FF3：引入“规模”与“价值”

**思想**：在市场因子之外，广泛数据中反复出现两种风格差异：小盘相对大盘、价值相对成长。把它们做成**可交易的多空组合**时间序列，纳入解释框架。  
**回归形式**：

$$
R_{i,t}-R_{f,t}
= \alpha_i
  + \beta_{i,M}(MKT_t - RF_t)
  + \beta_{i,SMB}SMB_t
  + \beta_{i,HML}HML_t
  + \varepsilon_{i,t}.
$$

* $MKT_t - RF_t=R_{M,t}-R_{f,t}$：市场超额收益。  
* $SMB_t$（Small Minus Big）：**小盘 − 大盘** 的多空组合当期回报；$\beta_{i,SMB}$ 是对“小盘相对大盘”这一差额的斜率。  
* $HML_t$（High Minus Low）：**高账面市值比 − 低账面市值比** 的多空组合回报；$\beta_{i,HML}$ 是对“价值相对成长”差额的斜率。  
* 直观解读：若 $\beta_{i,SMB}>0$，该资产更像“小盘风格”；若 $\beta_{i,HML}<0$，该资产更偏“成长风格”。

### FF5：再加入“盈利能力”与“投资”两个维度

**思想**：许多样本提示，**高盈利**公司往往有更高回报，而**低投资（更保守地扩张资产）**的公司在风险调整后回报更高。于是，在 FF3 的基础上加入两个因子。  
**回归形式**：

$$
R_{i,t}-R_{f,t}
= \alpha_i
  + \beta_{i,M}(MKT_t - RF_t)
  + \beta_{i,SMB}SMB_t
  + \beta_{i,HML}HML_t
  + \beta_{i,RMW}RMW_t
  + \beta_{i,CMA}CMA_t
  + \varepsilon_{i,t}.
$$

* $RMW_t$（Robust Minus Weak）：**高盈利 − 低盈利** 的多空组合回报；$\beta_{i,RMW}$ 是对“盈利维度差额”的斜率。  
* $CMA_t$（Conservative Minus Aggressive）：**低投资 − 高投资** 的多空组合回报；$\beta_{i,CMA}$ 是对“投资维度差额”的斜率。  
* **注意**：在 FF5 中，$SMB$ 的官方口径改为把三套分组（规模×价值、规模×盈利、规模×投资）得到的三个“小盘减大盘”序列取平均，以避免将规模与单一特征的耦合。

### 估计与解读

为便于对比与评分，建议采用如下统一做法：

1. **频率与口径**：使用月度总回报（含分红再投），所有序列按**月末**对齐；无风险利率与之同频。  
2. **回归**：以 OLS 估计各模型（CAPM、FF3、FF5），并使用固定滞后的 Newey–West 标准误（如滞后 $3$）。  
3. **报告与图表**：
   * 报告 $\alpha$、各 $\beta$ 与 $R^2$；比较 CAPM 与 FF3/FF5 的 $R^2$ 是否上升、$\alpha$ 是否下降。  
   * 绘制 $\beta$ 的条形对比图或滚动窗口折线，以观察风格暴露是否随时间漂移。  
4. **解读**：
   * 若加入风格因子后 $R^2$ 明显提升、$\alpha$ 明显接近 $0$，说明该资产的收益主要来自若干**共同影响因素**，而不是独立于这些因素的“额外回报”。  
   * 残差并不自动等同于“真正的阿尔法”；它也可能来自未纳入的风险因素、样本噪声或口径差异。

### 变量与符号速查

* $R_{i,t}$：资产 $i$ 在 $t$ 期的收益（建议用月度总回报）。  
* $R_{f,t}$：无风险利率（同频）。  
* $R_{M,t}$：按市值加权的“宽基市场”收益。  
* $MKT_t - RF_t=R_{M,t}-R_{f,t}$：市场超额收益。  
* $SMB_t$：小盘 − 大盘 的多空组合回报（FF3：源自“规模×价值”的分组；FF5：三套分组平均）。  
* $HML_t$：高账面市值比 − 低账面市值比 的多空组合回报。  
* $RMW_t$：高盈利 − 低盈利 的多空组合回报。  
* $CMA_t$：低投资 − 高投资 的多空组合回报。  
* $\alpha_i$：在控制了各因素后，资产 $i$ 的平均额外收益。  
* $\beta_{i,\cdot}$：资产对相应因素的**斜率/敏感度**（“该因素变动 $1\%$，该资产平均变化多少”）。  
* $\varepsilon_{i,t}$：特异性误差项。

### 与现实投资的联系

* **基金与产品评估**：拆分一只基金的历史超额收益，判断其来源更接近“大盘/风格”的顺风，还是策略本身（$\alpha$）。  
* **风格监测与风险预算**：跟踪组合在“规模、价值、盈利、投资”等维度的斜率，控制不希望集中的风格暴露。  
* **产品设计与绩效沟通**：将目标敞口写入投资说明（例如“长期保持小盘与价值偏好”），并用因子回归按季披露是否达成。  
* **再平衡**：当某一风格暴露偏离目标带宽时，通过调仓或对冲 ETF/期货因子篮子进行修正。